In [1]:
import numpy as np
import pandas as pd
import scipy
import json
from collections import Counter
import copy

In [2]:
%%time
# 입력데이터 로드
train = pd.read_json('train.json', typ = 'frame')
genre = pd.read_json('genre_gn_all.json', typ = 'series')
meta = pd.read_json('song_meta.json', typ = 'frame')

CPU times: total: 10.1 s
Wall time: 10.1 s


In [44]:
meta

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4
...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987


In [3]:
genre_df = genre.to_frame().reset_index()
genre_df = genre_df.rename(columns = {"index":"genre_code",0:"genre_name"})

genre_code_name_dict = dict()
for _, row in genre_df.iterrows():
    genre_code_name_dict[row["genre_code"]] = row["genre_name"]

#genre_code_name_dict

In [4]:
def mapping_function(data, col1, col2):
    # 플레이리스트 아이디(col1)와 수록곡(col2) 추출
    plylst_song_map = data[[col1, col2]]

    # unnest col2
    plylst_song_map_unnest = np.dstack(
        (
            np.repeat(plylst_song_map[col1].values, list(map(len, plylst_song_map[col2]))),
            np.concatenate(plylst_song_map[col2].values)
        )
    )

    # unnested 데이터프레임 생성 : plylst_song_map
    plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
    plylst_song_map[col1] = plylst_song_map[col1].astype(str)
    plylst_song_map[col2] = plylst_song_map[col2].astype(str)

    # unnest 객체 제거
    del plylst_song_map_unnest
    return plylst_song_map

In [97]:
playlst_song_map = mapping_function(train, 'id', 'songs')
agg = pd.DataFrame(playlst_song_map['songs'].value_counts()).reset_index()
agg.columns = ['곡', '플레이리스트내의 등장횟수']
meta['id'] = meta['id'].astype(str)
agg = agg.merge(meta[['id', 'song_name', 'song_gn_gnr_basket', 'artist_name_basket']], how='left', left_on='곡', right_on='id')
agg.drop(['곡','id'], axis=1, inplace=True)
agg = agg.rename(columns={'song_gn_gnr_basket':'genre_code', 'artist_name_basket': 'artist_name'})
agg = agg[agg["genre_code"].apply(lambda x: len(x) > 0)]

In [29]:
# 총 장르 수 254, 대분류 장르 30, 소분류 장르 224

gnr_code_list = list(genre.index) # 총 장르 list

gnr_code_name = pd.DataFrame(genre, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})
large_gnr_code_list = gnr_code_name[gnr_code_name['gnr_code'].str[-2:] == '00'] # 대분류 장르 list

c = Counter()
for gnr_code in meta["song_gn_gnr_basket"]:
    if len(gnr_code) == 0: continue
    for sub_item in gnr_code:
        if sub_item in gnr_code_list:
            c.update([sub_item])

gnrname_number = pd.DataFrame.from_dict(c, orient='index').reset_index().rename(columns = {0:'number','index':'genre_name'})
gnrcode_number = pd.DataFrame.from_dict(c, orient='index').reset_index().rename(columns = {0:'number','index':'genre_code'})
gnrname_number["genre_name"] = gnrname_number["genre_name"].apply(lambda code: genre[code])
gnrcode_number = gnrcode_number.sort_values(by=["number"], ascending=[False]).reset_index(drop=True)
gnrname_number = gnrname_number.sort_values(by=["number"], ascending=[False]).reset_index(drop=True)
gnrname_number.index += 1
#gnrname_number.head(10) # 장르별 곡 수 (장르 랭킹)
#gnrcode_number.head(10)
gnrname_10 = gnrname_number.head(10)
#gnrname_10


top10_genre_code = gnrcode_number["genre_code"][:10].tolist()

In [99]:
genre_song_dict = dict()
for genre_code in top10_genre_code:
    genre_song_dict.setdefault(genre_code, [])

total = 0
for i, row in agg.iterrows():
    genre_code_list = row["genre_code"]
    song_name = row["song_name"]
    artist_name = row["artist_name"]
    song_frequency = row["플레이리스트내의 등장횟수"]
    for g in genre_code_list:
        if not g in top10_genre_code: continue
        if len(genre_song_dict[g]) == 10: continue
        genre_song_dict[g].append(song_name + ' by ' + ','.join(artist_name) + ' with ' + str(song_frequency))
        total += 1
    if total == 100: break

#genre_song_dict

In [100]:
genre_top10_song = pd.DataFrame(genre_song_dict)
genre_top10_song.index = genre_top10_song.index+1
genre_top10_song = genre_top10_song.T.reset_index()
genre_top10_song = genre_top10_song.rename(columns = {'index':'genre_code'})
genre_top10_song["genre_code"] = genre_top10_song["genre_code"].apply(lambda code: genre_code_name_dict[code])
genre_top10_song.index = genre_top10_song.index+1
genre_top10_song = genre_top10_song.rename(columns = {'genre_code': 'genre_name'})
#genre_top10_song

In [102]:
merged_df = genre_top10_song.merge(gnrname_10, on='genre_name', how='left')

desired_columns = ['genre_name', 'number']+[i for i in range(1,11)]
merged_df = merged_df[desired_columns]
merged_df.index = merged_df.index+1
merged_df

,genre_name,number,1,2,3,4,5,6,7,8,9,10
1,POP,77101,Lost Stars by Adam Levine with 1452,I`m Not The Only One by Sam Smith with 1421,Shape of You by Ed Sheeran with 1306,YOUTH by Troye Sivan with 1190,Sugar by Maroon 5 with 1138,Uptown Funk (Feat. Bruno Mars) by Mark Ronson ...,Love Yourself by Justin Bieber with 1108,Happy (Gru`s Theme From Despicable Me 2) by Ph...,One Call Away by Charlie Puth with 970,Sunday Morning by Maroon 5 with 926
2,록/메탈,63926,Lost Stars by Adam Levine with 1452,Thinking Out Loud by Ed Sheeran with 1175,Sugar by Maroon 5 with 1138,Fix You by Coldplay with 829,Counting Stars by OneRepublic with 770,The Scientist by Coldplay with 762,Photograph by Ed Sheeran with 730,Lucky (Feat. Colbie Caillat) by Jason Mraz wit...,Lucky Strike by Maroon 5 with 687,Everglow by Coldplay with 687
3,일렉트로니카,60144,Closer (Feat. Halsey) by The Chainsmokers with...,"Something Just Like This by The Chainsmokers,C...",The Ocean (Feat. Shy Martin) (Radio Edit.) by ...,This Is What You Came For (Feat. Rihanna) by C...,Faded by Alan Walker with 845,How To Love (Feat. Sofia Reyes) by Cash Cash w...,"Feels (Feat. Pharrell Williams, Katy Perry & B...",Get Lucky (Feat. Pharrell Williams) by Daft Pu...,Bad (Feat. Vassy) (Radio Edit) by David Guetta...,Slide (Feat. Frank Ocean & Migos) by Calvin Ha...
4,클래식,57066,Debussy : Reverie L.68 (드뷔시 : 꿈) by Various Ar...,Provance by Takakuwa Hideyo with 168,Grasse by Takakuwa Hideyo with 111,Mozart : Piano Sonata No.9 In D Major K.311 - ...,Kreisler - Tambourin Chinois Op.3 (크라이슬러 : 중국의...,Kreisler - Kreisler Rachmaninov Liebesfreud (크...,Liszt : Grandes Etudes De Paganini No.6 In A M...,Satie : Gymnopedies No.1 - Lent Et Douloureux ...,10월의 어느 멋진 날에 by 김동규 with 91,Debussy : Children`s Corner L.113 - II. Jimbo`...
5,발라드,56820,밤편지 by 아이유 with 2175,어떤이별 by 임승부 with 1599,야생화 by 박효신 with 1598,비가 내렸어 (Vocal by 스티브언니) by 업라이트 (Upright) with...,벙어리 by 홍아 with 1540,쉬운사랑 by 중신 with 1538,고마운 사람 (Vocal by 이소진) by 업라이트 (Upright) with 1508,"우연히 봄 by 로꼬,유주 (여자친구) with 1507",금요일에 만나요 (Feat. 장이정 Of HISTORY) by 아이유 with 1501,선물 by 민우 with 1492
6,재즈,48199,Don`t Know Why by Norah Jones with 795,Breezy by Wouter Hamel with 531,Home by Michael Buble with 495,"Quando, Quando, Quando by Michael Buble with 391",Sleeping Beauty by Paul with 380,Soul Nights by Music Shower with 371,Chocolate Drive by 모이다밴드 with 325,I Wish You Love by Lisa Ono with 315,L - O - V - E by Olivia Ong with 308,"March, April, May by Wouter Hamel with 299"
7,OST,44272,눈의 꽃 by 박효신 with 1647,"우연히 봄 by 로꼬,유주 (여자친구) with 1507",Lost Stars by Adam Levine with 1452,너였다면 by 정승환 with 1354,너의 모든 순간 by 성시경 with 1215,첫눈처럼 너에게 가겠다 by 에일리 with 1207,너무 보고싶어 by 어쿠스틱 콜라보 with 1207,"모든 날, 모든 순간 (Every day, Every Moment) by 폴킴 wi...",만약에 by 태연 (TAEYEON) with 1184,걱정말아요 그대 by 이적 with 1152
8,뉴에이지,42515,Blue Sea by Bird with 834,Happy Day by Green Diary with 713,따뜻한 그날 by 우리집 with 679,Last Carnival by Acoustic Cafe with 653,Fresh Morning by Healing with 647,River Flows In You by 이루마 with 623,바람따라 가는 기차여행 by 감성노트 with 613,This Morning by 제이 리 with 611,Daily Life by Green Tea with 609,Suddenly by 제이 리 with 597
9,인디음악,42468,안아줘 by 정준일 with 2121,가끔 미치도록 네가 안고 싶어질 때가 있어 by 가을방학 with 1919,"그대와 나, 설레임 (Feat. 소울맨) by 어쿠스틱 콜라보 with 1633",널 생각해 by 원 모어 찬스 (one more chance) with 1538,선물 by 민우 with 1492,사랑을 놓치고 by 박은옥 with 1476,넌 이별 난 아직 (With 한소현 Of 3rd Coast) by 스탠딩 에그 wi...,스토커 by 10CM with 1468,"수고했어, 오늘도 by 옥상달빛 with 1446",불안해 by 혜지 (Hyeji) with 1434
10,랩/힙합,35376,"See You Again (폴 워커 추모 엔딩곡) by Charlie Puth,Wi...","I`m The One (Feat. Justin Bieber, Quavo, Chanc...",Nothin` On You (Feat. Bruno Mars) by B.o.B wit...,Empire State Of Mind (Feat. Alicia Keys) by JA...,GDFR (Feat. Sage The Gemini And Lookas) by Flo...,HUMBLE. by Kendrick Lamar with 537,Congratulations (Feat. Quavo) by Post Malone w...,Caroline by Amine with 451,The Monster (Feat. Rihanna) by Eminem with 442,Panda by Desiigner with 440
